# KPIer

- Vaccination över tid och påverkan på dödsfall och intensivvård: Minskade dödsfall och svårt sjuka med vaccination?
- Jämförelse av fall i olika kommuner: Var det värst i de största städerna?
- Intensivvård mot dödsfall: Hur effektiv var vården?
- Åldersskillnaden i fall: Var valet att vaccinera äldre först rätt?



## Explorativ analys

Började med en manuell lätt avläsning av de olika excel bladen för att få en idé av vilken data som fanns och vilka KPIer som skulle kunna göra från den.

Bladed vaccinationer tidsserie ger en bild av antalet vaccinationer över tid vilket skulle kunna jämnföras med olika coviddata som dödsfall för att få en idé om hur effektivt vaccinet var. 

Det fanns också olika data från kommuner både kring vaccinationer och fall, möjliga undersökningar som skulle kunna göras är om det var värre i storstäderna mot landsbyggden och om sättet att fördela vacciner var rättvist/effektivt. Intressant sidopunkt att se om det fanns någon kommun som hade många fall men dålig vaccinationsmängd.

Jag hittade också olika intensivvårdsdata som skulle kunna jämföras med dödsfallen för att få en bild av hur effektiv intensivvården var, får göra mer dataanalys för att se om det fanns data för avlidna i vården eller om det enbart går att jämnföra dödsfall.

En till sak att titta på är åldersmätningarna som fanns och hur distributionen där ser ut, om valet att vaccinera äldre först rätt.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px


df_timeframe = pd.read_excel("Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name= "Vaccinationer tidsserie")
df_timeframe.head()

,Vecka,År,Region,Antal vaccinationer
0,52,2020,| Sverige |,2478
1,52,2020,Stockholm,289
2,52,2020,Uppsala,9
3,52,2020,Södermanland,116
4,52,2020,Östergötland,214


In [2]:
# Data on number of vaccinations divided into week/year/Region note that region does contain a region representing the whole country
df_timeframe.info()
df_timeframe.value_counts()
df_timeframe["Region"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Vecka                2134 non-null   int64 
 1   År                   2134 non-null   int64 
 2   Region               2134 non-null   object
 3   Antal vaccinationer  2134 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 66.8+ KB


array(['| Sverige |', 'Stockholm', 'Uppsala', 'Södermanland',
       'Östergötland', 'Jönköping', 'Kronoberg', 'Kalmar', 'Gotland',
       'Blekinge', 'Skåne', 'Halland', 'Västra Götaland', 'Värmland',
       'Örebro', 'Västmanland', 'Dalarna', 'Gävleborg', 'Västernorrland',
       'Jämtland', 'Västerbotten', 'Norrbotten'], dtype=object)

### Notering
Eftersom jag vill jämnföra denna datan men dödsfall i riket och det är per vecka så läser jag in bladed veckodata riket. Jag har redan gjort en dataanalys av detta bladet i uppgift 1 och kommer därför inte göra det här.

In [3]:
# usefull columns år, veckonummer, Antal_nyaintensivvårdade_vecka, Antal_avlidna_vecka
df_cases_country = pd.read_excel("Folkhalsomyndigheten_Covid19.xlsx", sheet_name= "Veckodata Riket")
df_cases_country.head()

,år,veckonummer,Antal_fall_vecka,Antal_fall_100000inv_vecka,Antal_fall_100000inv_14dagar,Kum_antal_fall,Kum_fall_100000inv,Antal_nyaintensivvårdade_vecka,Kum_antal_intensivvårdade,Antal_avlidna_vecka,Antal_avlidna_milj_inv_vecka,Kum_antal_avlidna,Kum_antal_avlidna_milj_inv
0,2020,6,1,0,0,1,0,0,0,0,0.0,0,0.0
1,2020,7,0,0,0,1,0,0,0,0,0.0,0,0.0
2,2020,8,0,0,0,1,0,0,0,0,0.0,0,0.0
3,2020,9,13,0,0,14,0,0,0,0,0.0,0,0.0
4,2020,10,211,2,2,225,2,3,3,0,0.0,0,0.0


In [4]:
# cleaning up dataframe so that it only has data needed
df_timeframe = df_timeframe[df_timeframe["Region"] == "| Sverige |"]
df_timeframe["Vecka"] = df_timeframe["År"].astype(str) + "v" + df_timeframe["Vecka"].astype(str)
# could drop "Region" aswell here
df_timeframe.drop("År", axis = 1, inplace=True)
df_timeframe.reset_index(drop=True, inplace= True)
df_timeframe

,Vecka,Region,Antal vaccinationer
0,2020v52,| Sverige |,2478
1,2020v53,| Sverige |,16581
2,2021v1,| Sverige |,53258
3,2021v2,| Sverige |,132915
4,2021v3,| Sverige |,249713
...,...,...,...
92,2022v39,| Sverige |,24260220
93,2022v40,| Sverige |,24469813
94,2022v41,| Sverige |,24641177
95,2022v42,| Sverige |,24778109


In [10]:
df_cases_country["Vecka"] = df_cases_country["år"].astype(str) + "v" + df_cases_country["veckonummer"].astype(str)
df_cases_country.drop(df_cases_country.columns.difference(["Vecka", "Antal_nyaintensivvårdade_vecka", "Antal_avlidna_vecka"]), axis = 1, inplace=True)
df_cases_country

,Antal_nyaintensivvårdade_vecka,Antal_avlidna_vecka,Vecka
0,0,0,2020v6
1,0,0,2020v7
2,0,0,2020v8
3,0,0,2020v9
4,3,0,2020v10
...,...,...,...
138,17,94,2022v39
139,20,98,2022v40
140,15,94,2022v41
141,20,70,2022v42
